# Example of grouped regressions

In this section, we want to demanstrate a slightly advanced example for using Pandas grouped transformation for performing many ordinary least square model fits in parallel. We reuse the weather data and try to predict the temperature of all stations with a very simple model per station.

In [ ]:
%matplotlib inline

# 1 Load Data
First we load data of a single year.

In [ ]:
storageLocation = "s3://dimajix-training/data/weather"

In [ ]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

rawWeatherData = spark.read.text(storageLocation + "/2003")
weather_all = rawWeatherData.select(
    substring(col("value"),5,6).alias("usaf"),
    substring(col("value"),11,5).alias("wban"),
    to_timestamp(substring(col("value"),16,12),"yyyyMMddHHmm").alias("timestamp"),
    to_timestamp(substring(col("value"),16,12),"yyyyMMddHHmm").cast("long").alias("ts"),
    substring(col("value"),42,5).alias("report_type"),
    substring(col("value"),61,3).alias("wind_direction"),
    substring(col("value"),64,1).alias("wind_direction_qual"),
    substring(col("value"),65,1).alias("wind_observation"),
    (substring(col("value"),66,4).cast("float") / lit(10.0)).alias("wind_speed"),
    substring(col("value"),70,1).alias("wind_speed_qual"),
    (substring(col("value"),88,5).cast("float") / lit(10.0)).alias("air_temperature"),
    substring(col("value"),93,1).alias("air_temperature_qual")
)

# 2 Analysis of one station

First we only analyse a single station, just to check our approach and the expressiveness of our model. It won't be a very good fit, but it will be good enough for our needs to demonstrate the concept.

So first we pick a single station, and we also only keep those records with a valid temeprature measurement.

In [ ]:
weather_single = weather_all.where("usaf='954920' and wban='99999'").cache()

In [ ]:
pdf = # YOUR CODE HERE
pdf

## 2.1 Create Feature Space

Our model will simply predict the temperature depending on the time and day of year. We use sin and cos of with a day-wide period and a year-wide period as features for fitting the model.

In [ ]:
import numpy as np
import math

seconds_per_day = 24*60*60
seconds_per_year = 365*seconds_per_day

# Add sin and cos as features for fitting
pdf['daily_sin'] = np.sin(pdf['ts']/seconds_per_day*2.0*math.pi)
pdf['daily_cos'] = np.cos(pdf['ts']/seconds_per_day*2.0*math.pi)
pdf['yearly_sin'] = np.sin(pdf['ts']/seconds_per_year*2.0*math.pi)
pdf['yearly_cos'] = np.cos(pdf['ts']/seconds_per_year*2.0*math.pi)

# Make a plot, just to check how it looks like
pdf[0:200].plot(x='timestamp', y=['daily_sin','daily_cos','air_temperature'], figsize=[16,6])

## 2.2 Fit model

Now that we have the temperature and some features, we fit a simple model.

In [ ]:
import statsmodels.api as sm

# define target variable y
y = pdf['air_temperature']
# define feature variables X
X = pdf[['ts', 'daily_sin', 'daily_cos', 'yearly_sin', 'yearly_cos']]
X = sm.add_constant(X)
# fit model
model = sm.OLS(y, X).fit()

# perform prediction
pdf['pred'] = model.predict(X)

# Make a plot of real temperature vs predicted temperature
pdf[0:200].plot(x='timestamp', y=['pred','air_temperature'], figsize=[16,6])

## 2.3 Inspect Model

Now let us inspect the model, in order to find a way to store it in a Pandas DataFrame

In [ ]:
# YOUR CODE HERE

In [ ]:
type(model.params)

Finally let us create a Pandas DataFrame from the model parameters. This code snippet will be needed later when we want to parallelize the fitting for different weather stations using Spark.

In [ ]:
x_columns = X.columns
pd.DataFrame([[model.params[i] for i in  x_columns]], columns=x_columns)

# 3 Perform OLS for all stations

Now we want to create a model for all stations. First we filter the data again, such that we only have valid temperature measurements.

In [ ]:
valid_weather = weather_all.filter(weather_all.air_temperature_qual == 1)

## 3.1 Feature extraction

Now we generate the same features, but this time we use Spark instead of Pandas operations. This simplifies later model fitting.

In [ ]:
import math

seconds_per_day = 24*60*60
seconds_per_year = 365*seconds_per_day

features = valid_weather.select(
    valid_weather.usaf,
    valid_weather.wban,
    valid_weather.air_temperature,
    valid_weather.ts,
    lit(1.0).alias('const'),
    sin(valid_weather.ts * 2.0 * math.pi / seconds_per_day).alias('daily_sin'),
    cos(valid_weather.ts * 2.0 * math.pi / seconds_per_day).alias('daily_cos'),
    sin(valid_weather.ts * 2.0 * math.pi / seconds_per_year).alias('yearly_sin'),
    cos(valid_weather.ts * 2.0 * math.pi / seconds_per_year).alias('yearly_cos')
)

features.limit(10).toPandas()

## 3.2 Fit Models

Now we use a Spark Pandas grouped UDF in order to fit models for all weather stations in parallel.

In [ ]:
group_columns = ['usaf', 'wban']
y_column = 'air_temperature'
x_columns = ['ts', 'const', 'daily_sin', 'daily_cos', 'yearly_sin', 'yearly_cos']
schema = features.select(*group_columns, *x_columns).schema

@pandas_udf(schema, PandasUDFType.GROUPED_MAP)
def ols(pdf):
    # Extract grouping information from appropriate columns
    group = # YOUR CODE HERE
    
    # Extract target variable
    y = # YOUR CODE HERE
    
    # Extract predictor variables
    X = # YOUR CODE HERE
    
    # Create model using Python statsmodel package to fit y to input variables x
    model = sm.OLS(y, X).fit()
    
    # Create a Pandas data frame with one row containing the grouping columns and all model parameters
    return pd.DataFrame([group + [model.params[i] for i in x_columns]], columns=group_columns + x_columns)

# Now fit model for all weather stations in parallel using Spark
models = # YOUR CODE HERE

In [ ]:
models.limit(10).toPandas()

## 3.3 Inspect and compare results

Now let's pick the same station again, and compare the model to the original model.

In [ ]:
models.where("usaf='954920' and wban='99999'").toPandas()

In [ ]:
model.params